In [1]:
import os
import shutil
import random
from glob import glob
import xml.etree.ElementTree as ET

from sklearn.model_selection import train_test_split

In [2]:
CLASSES_PATH = '../class_names/classes.txt'

'''loads the classes'''
def get_classes(classes_path):
    with open(classes_path) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    return class_names


classes = get_classes(CLASSES_PATH)
assert len(classes) > 0, 'no class names detected!'
print(f'num classes: {len(classes)}')

num classes: 1


In [3]:
def write_xml_2_anno_txt_file(TXT_PATH, path):
    ## output file
    list_file = open(TXT_PATH, 'a')
    
    # path    = '../../img/croped_1s/a_000003.xml'
    in_file = open(path)
    
    ## Parse .xml file
    tree = ET.parse(in_file)
    root = tree.getroot()
    
    ## Write object information to .txt file
    file_name = root.find('filename').text
    list_file.write(file_name)
    for obj in root.iter('object'):
        cls    = obj.find('name').text                 
        cls_id = classes.index(cls)
        xmlbox = obj.find('bndbox')
        b      = (int(xmlbox.find('xmin').text), int(xmlbox.find('ymin').text), int(xmlbox.find('xmax').text), int(xmlbox.find('ymax').text))
        list_file.write(" " + ",".join([str(a) for a in b]) + ',' + str(cls_id))
    list_file.write('\n')
    list_file.close()




In [4]:
def copy_img(source, destin, img_name):
    # copy image
    source = source + img_name + '.png'
    destin = destin + img_name + '.png'
    shutil.copyfile(source, destin)   

    # # copy xml
    # source = r'../../img/croped_1s/{}.xml'.format(img_name)
    # destin = r'../dataset/train_vali_xml_sopbox/{}.xml'.format(img_name)
    # try:
    #     shutil.copyfile(source, destin) 
    # except FileNotFoundError:
    #     with open(r'../dataset/txt/.txt','a') as f:
    #         f.write(img_name + '.png\n')
    #     print("{} not have xml".format(img_name))
    # except:
    #     print("other error")

# 找出有label(壺腹乳頭)的圖片

In [5]:
xml_paths = glob("../../img/croped_1s/*.xml")
haveLabels = []
for xml_path in xml_paths:
    img_name = xml_path.split('\\')[1].split('.')[0]
    haveLabels.append(img_name)

print('有label(壺腹乳頭)的圖片數量= ', len(haveLabels))

有label(壺腹乳頭)的圖片數量=  305


# 將有label(壺腹乳頭)的圖片分成train/vali/test

### 複製 train/vali 影像 至 dataset/train_vali_img_sopbox 資料夾
### 複製 test 影像 至 test_img/test_img_sopbox 資料夾

In [6]:
XML_PATH       = '../../img/croped_1s/'
train_TXT_PATH = '../dataset/txt/20220818_anno_train_lines_0.txt'     # !! txt要先創
vali_TXT_PATH  = '../dataset/txt/20220818_anno_val_lines_0.txt'       # !! txt要先創
test_TXT_PATH  = '../test_img/txt/20220818_anno_test_lines_0.txt'     # !! txt要先創
train = ['a', 'b', 'c', 'e', 'f','g', 'j']
vali  = ['d', 'i']
test  = ['h', 'z']

for img_name in haveLabels:
    if img_name[0] in train:
        path = XML_PATH + img_name + '.xml'
        print(path)
        write_xml_2_anno_txt_file(train_TXT_PATH, path)
        copy_img(source=XML_PATH, destin='../dataset/20220818_train_vali_img_sopbox_0/', img_name=img_name)
    
    elif img_name[0] in vali:
        path = XML_PATH + img_name + '.xml'
        print(path)
        write_xml_2_anno_txt_file(vali_TXT_PATH, path)
        copy_img(source=XML_PATH, destin='../dataset/20220818_train_vali_img_sopbox_0/', img_name=img_name)

    if img_name[0] in test:
        path = XML_PATH + img_name + '.xml'
        print(path)
        write_xml_2_anno_txt_file(test_TXT_PATH, path)
        copy_img(source=XML_PATH, destin='../test_img/20220818_test_img_sopbox_0/', img_name=img_name)

../../img/croped_1s/a_000001.xml
../../img/croped_1s/a_000002.xml
../../img/croped_1s/a_000003.xml
../../img/croped_1s/a_000004.xml
../../img/croped_1s/a_000005.xml
../../img/croped_1s/a_000006.xml
../../img/croped_1s/a_000007.xml
../../img/croped_1s/a_000008.xml
../../img/croped_1s/a_000009.xml
../../img/croped_1s/a_000010.xml
../../img/croped_1s/b_000051.xml
../../img/croped_1s/b_000052.xml
../../img/croped_1s/b_000053.xml
../../img/croped_1s/b_000054.xml
../../img/croped_1s/b_000055.xml
../../img/croped_1s/b_000056.xml
../../img/croped_1s/b_000057.xml
../../img/croped_1s/b_000058.xml
../../img/croped_1s/c_000035.xml
../../img/croped_1s/c_000036.xml
../../img/croped_1s/c_000037.xml
../../img/croped_1s/c_000038.xml
../../img/croped_1s/c_000039.xml
../../img/croped_1s/c_000040.xml
../../img/croped_1s/c_000041.xml
../../img/croped_1s/c_000042.xml
../../img/croped_1s/c_000043.xml
../../img/croped_1s/c_000044.xml
../../img/croped_1s/c_000045.xml
../../img/croped_1s/c_000046.xml
../../img/

# 找出沒有label(壺腹乳頭)的圖片
#### 隨機挑選10%沒有壺腹乳頭圖片 加入訓練集 
#### 所有 沒有壺腹乳頭圖片的測試資料 加入測試集

In [ ]:
img_paths = glob("../../img/croped_1s/*.png")
img_names = []
for img_path in img_paths:
    img_name = img_path.split('\\')[1].split('.')[0]
    img_names.append(img_name)


unhaveLabels = []
for img_name in img_names:
    if img_name not in haveLabels:
        unhaveLabels.append(img_name)

len(unhaveLabels)

In [ ]:
# 利用train_test_split完成隨機挑選
X = []
y = []
test_unhaveLabels = []
for img_name in unhaveLabels:
    if img_name[0] in train:
        X.append(img_name)
        y.append(img_name[0])
    elif img_name[0] in vali:
        X.append(img_name)
        y.append(img_name[0])
    elif img_name[0] in test:
        test_unhaveLabels.append(img_name)

_,unhaveLabels_random,_,_ = train_test_split(X,y,test_size=0.1,stratify=y,random_state=5)

In [ ]:
len(test_unhaveLabels)

In [ ]:
len(unhaveLabels_random)

In [ ]:
def write_nothing_2_anno_txt_file(TXT_PATH, img_name):
    '''寫img_name到anno.txt file
    '''
    ## output file
    list_file = open(TXT_PATH, 'a')
    list_file.write(img_name)
    list_file.write('\n')
    list_file.close()

#### 將沒有label(壺腹乳頭)的圖片複製進 train/test 影像 至 test_img/test_img_sopbox_0(1) 資料夾

In [ ]:
for img_name in unhaveLabels_random:
    if img_name[0] in train:
        write_nothing_2_anno_txt_file(train_TXT_PATH, img_name+'.png')
        copy_img(source=XML_PATH, destin='../dataset/train_vali_img_sopbox_1/', img_name=img_name)
    
    elif img_name[0] in vali:
        write_nothing_2_anno_txt_file(vali_TXT_PATH, img_name+'.png')
        copy_img(source=XML_PATH, destin='../dataset/train_vali_img_sopbox_1/', img_name=img_name)
    
    
for img_name in test_unhaveLabels:
    if img_name[0] in test: 
        write_nothing_2_anno_txt_file(test_TXT_PATH, img_name+'.png')
        copy_img(source=XML_PATH, destin='../test_img/test_img_sopbox_1/', img_name=img_name)

製作資料集

In [ ]:
# X = haveLabels + unhaveLabels
# y = [0]*len(haveLabels) + [1]*len(unhaveLabels)

# print(len(X))

## 分割資料集成訓練集&測試集

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,stratify=y)

# print(len(X_train),
#       len(X_test),
#       len(y_train),
#       len(y_test))

複製測試集至Yolov4 test_img 資料夾

In [ ]:
# for img_name in X_test:

#     # copy xml
#     source = r'../../img/croped_1s/{}.xml'.format(img_name)
#     destin = r'../test_img/X_test_big_56/{}.xml'.format(img_name)
#     try:
#         shutil.copyfile(source, destin) 
#         # copy image
#         source_ = r'../../img/croped_1s/{}.png'.format(img_name)
#         destin_ = r'../test_img/X_test_big_56/{}.png'.format(img_name)
#         shutil.copyfile(source_, destin_)

#     except FileNotFoundError:
#         print("{} not have xml".format(img_name))
#     except:
#         print("other error")
       